# Projeto 3 - Classificando clientes em grupos

### Pegando dados

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

## Colunas dos dados

* CUSTID: Identificação do Titular do Cartão de Crédito
* SALDO: Saldo restante na conta do cliente para efetuar compras
* BALANCE_FREQUENCY: Com que frequência o Saldo é atualizado, pontuação entre 0 e 1 (1 = atualizado com frequência, 0 = não atualizado com frequência)
* COMPRAS: Valor de compras feitas na conta
* ONEOFFPURCHASES: Valor máximo de compra feito de uma só vez
* INSTALLMENTS_PURCHASES: Valor da compra parcelada
* CASH_ADVANCE: Dinheiro adiantado dado pelo usuário
* PURCHASES_FREQUENCY: Com que frequência as Compras estão sendo feitas, pontue entre 0 e 1 (1 = compra frequentemente, 0 = não compra com frequência)
* ONEOFF_PURCHASES_FREQUENCY: Com que frequência as compras acontecem de uma só vez (1 = comprado com frequência, 0 = não comprado com frequência)
* PURCHASES_INSTALLMENTS_FREQUENCY: Com que frequência as compras parceladas estão sendo feitas (1 = feito com frequência, 0 = não feito com frequência)
* CASH_ADVANCE_FREQUENCY: Com que frequência o dinheiro adiantado está sendo pago
* CASH_ADVANCE_TRX: Quantidade de Transações feitas com "Dinheiro Adiantado"
* PURCHASES_TRX: Número de transações de compra realizadas
* CREDIT_LIMIT: Limite de Cartão de Crédito por usuário
* PAGAMENTOS: Valor do pagamento feito pelo usuário
* MINIMUM_PAYMENTS: Valor mínimo de pagamentos feitos pelo usuário
* PRC_FULL_PAYMENT: Porcentagem do pagamento integral pago pelo usuário
* TENURE: Posse do serviço de cartão de crédito para o usuário

In [ ]:
dados_clientes = pd.read_csv('dados_clientes.csv')

In [ ]:
dados_clientes = dados_clientes.drop('CUST_ID', axis = 1) #vamos deixar nosso DF apenas com números

In [ ]:
dados_clientes = dados_clientes.dropna()
dados_clientes = dados_clientes.reset_index(drop = True)

dados_clientes

# Achando número ótimos de grupos - Elbow Method na prática.

In [ ]:
escalador = StandardScaler()
dados_clientes_escalados = escalador.fit_transform(dados_clientes)

dados_clientes_escalados

In [ ]:
dados_clientes_escalados.shape

In [ ]:
#Vamos aplicar o elbow antes de rodar de vez os grupos finais.

lista_WCSS = []

range_de_grupos = range(1, 50)

for i in range_de_grupos:
  kmeans = KMeans(n_clusters = i)
  kmeans.fit(dados_clientes_escalados)
  lista_WCSS.append(kmeans.inertia_) 

lista_WCSS

In [ ]:
plt.plot(lista_WCSS)
plt.title('Achando o número ótimo de clusters')
plt.xlabel('Clusters')
plt.ylabel('WCSS') 
plt.show()

# Criando nossos grupos definitivos K-MEANS

In [ ]:
kmeans = KMeans(10)
kmeans.fit(dados_clientes_escalados)
grupos = kmeans.labels_

grupos

In [ ]:
centro_dos_dados_por_cluster = kmeans.cluster_centers_
centro_dos_dados_por_cluster = escalador.inverse_transform(centro_dos_dados_por_cluster)

In [ ]:
centro_dos_dados_por_cluster = pd.DataFrame(data = centro_dos_dados_por_cluster, columns = [dados_clientes.columns])
centro_dos_dados_por_cluster  

### Juntando os grupos no df original

In [ ]:
dados_clientes_com_grupos = pd.concat([dados_clientes, pd.DataFrame({'grupo':grupos})], axis = 1)
dados_clientes_com_grupos

In [ ]:
#visualizando dados por cluster

for i in dados_clientes_com_grupos.columns:
  plt.figure(figsize = (35, 5))
  for j in range(10):
    plt.subplot(1,10,j+1)
    grupo = dados_clientes_com_grupos[dados_clientes_com_grupos['grupo'] == j]
    grupo[i].hist(bins = 20)
    plt.title(f'{i}    \nGrupo {j} ')
  
  plt.show()

# Usando o PCA para verificar se o K-MEANS fez um bom trabalho

In [ ]:
pca = PCA(n_components=2)
dados_clientes_pca = pca.fit_transform(dados_clientes_escalados)
dados_clientes_pca

In [ ]:
pca_df = pd.DataFrame(data = dados_clientes_pca, columns =['pca1','pca2'])
pca_df

In [ ]:
pca_df = pd.concat([pca_df,pd.DataFrame({'grupo':grupos})], axis = 1)
pca_df

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.scatterplot(x="pca1", y="pca2", hue = "grupo", 
                     data = pca_df, palette =['red','green','blue','pink','yellow',
                                              'gray','purple', 'black', 'orange', 'navy'])
plt.show()